In [4]:
from collections.abc import Iterable
import os

from dandi.dandiapi import DandiAPIClient
from datetime import datetime, timedelta
from dateutil import tz
import h5py
from hdmf.backends.hdf5.h5_utils import H5DataIO
from hdmf.container import Container
from hdmf.data_utils import DataChunkIterator
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pynwb import load_namespaces, get_class, register_class, NWBFile, TimeSeries, NWBHDF5IO
from pynwb.file import MultiContainerInterface, NWBContainer, Device, Subject
from pynwb.ophys import ImageSeries, OnePhotonSeries, OpticalChannel, ImageSegmentation, PlaneSegmentation, Fluorescence, DfOverF, CorrectedImageStack, MotionCorrection, RoiResponseSeries, ImagingPlane
from pynwb.core import NWBDataInterface
from pynwb.epoch import TimeIntervals
from pynwb.behavior import SpatialSeries, Position
from pynwb.image import ImageSeries
import remfile
import scipy.io as sio
import skimage.io as skio
from tifffile import TiffFile

# ndx_mulitchannel_volume is the novel NWB extension for multichannel optophysiology in C. elegans
#from ndx_multichannel_volume import CElegansSubject, OpticalChannelReferences, OpticalChannelPlus, ImagingVolume, VolumeSegmentation, MultiChannelVolume, MultiChannelVolumeSeries

In [11]:
import os
import PyQt6.QtCore
os.environ["QT_API"] = "pyqt6"

## Streaming from Dandi tutorial

Here we provide an example for streaming files from an online Dandi set. Dandi allows you to selectively read in only specific data objects or chunks, allowing you to access parts of files without having to load the whole object into memory. Here we provide an example of extracting neural activity traces from dandiset 000472 and visualizing them in both the time and frequency domains. You can find more information at https://pynwb.readthedocs.io/en/stable/tutorials/advanced_io/streaming.html.

In [10]:
dandi_id = '000472' #EY dataset

AVA_traces = []
SMDV_traces = []
AWC_traces = []
ASH_traces = []
RID_traces = []

with DandiAPIClient() as client:
    dandiset = client.get_dandiset(dandi_id, 'draft')
    for asset in dandiset.get_assets():
        s3_url = asset.get_content_url(follow_redirects=1, strip_query=True)
        file = remfile.File(s3_url)

        with h5py.File(file, 'r') as f:
            with NWBHDF5IO(file=f, mode='r', load_namespaces=True) as io:
                read_nwb = io.read()
                identifier = read_nwb.identifier
                print(identifier)
                seg = read_nwb.processing['NeuroPAL']['NeuroPALSegmentation']['NeuroPALNeurons'].voxel_mask[:]
                labels = read_nwb.processing['NeuroPAL']['NeuroPALSegmentation']['NeuroPALNeurons']['ID_labels'][:]
                rate = read_nwb.acquisition['CalciumImageSeries'].rate
                fluor = read_nwb.processing['CalciumActivity']['SignalRawFluor']['SignalCalciumImResponseSeries'].data[:]
                calc_labels = read_nwb.processing['CalciumActivity']['NeuronIDs'].labels[:]
                scale = read_nwb.imaging_planes['NeuroPALImVol'].grid_spacing[:]
        
        index_AVA = np.argwhere((np.asarray(calc_labels)=='AVAR') | (np.asarray(calc_labels)=='AVAL'))
        index_SMDV = np.argwhere((np.asarray(calc_labels)=='SMDVR') | (np.asarray(calc_labels)=='SMDVL'))
        index_AWC = np.argwhere((np.asarray(calc_labels)=='AWCR') | (np.asarray(calc_labels)=='AWCL'))
        index_ASH = np.argwhere((np.asarray(calc_labels)=='ASHR') | (np.asarray(calc_labels)=='ASHL'))
        index_RID = np.argwhere(np.asarray(calc_labels)=='RID')
        AVA_traces = AVA_traces + [np.squeeze(fluor[:,index]/np.mean(fluor[:,index])) for index in index_AVA if index is not None]
        SMDV_traces = SMDV_traces + [np.squeeze(fluor[:,index]/np.mean(fluor[:,index])) for index in index_SMDV if index is not None]
        AWC_traces = AWC_traces + [np.squeeze(fluor[:, index]/np.mean(fluor[:,index])) for index in index_AWC if index is not None]
        ASH_traces = ASH_traces + [np.squeeze(fluor[:,index]/np.mean(fluor[:,index])) for index in index_ASH if index is not None]
        RID_traces = RID_traces + [np.squeeze(fluor[:,index]/np.mean(fluor[:,index])) for index in index_RID if index is not None]
        

2022-02-11-w03-NP1
2021-12-03-w00-NP1
2022-02-12-w00-NP1
2022-02-12-w01-NP1
2022-01-22-w04-NP1
2022-02-22-w04-NP1
2022-04-01-w00-NP1
2022-03-05-w00-NP1
2022-04-26-w01-NP1
2022-04-26-w00-NP1


In [5]:
%matplotlib qt

def plot_traces_fourier(traces, labels, rate):

    fig, axs = plt.subplots(len(traces),2)

    for i, neuron_trace in enumerate(traces):
        for j, trace in enumerate(neuron_trace):
            trace_cropped = trace[:936] #truncate so all traces have the same length
            time_steps = np.linspace(0,936/4.0,936)
            axs[i,0].plot(time_steps,trace_cropped, alpha=0.5)

            n = len(trace_cropped)
            yf = np.fft.rfft(trace_cropped)
            xf = np.fft.rfftfreq(n, d=1/rate)

            axs[i,1].plot(xf, yf, alpha=0.5)
        
        axs[i,0].set_title('Time domain - '+labels[i])
        axs[i,1].set_title('Frequency domain - '+labels[i])

traces = [AVA_traces, SMDV_traces, AWC_traces, ASH_traces, RID_traces]
labels = ['AVA', 'SMDV', 'AWC', 'ASH', 'RID']
plot_traces_fourier(traces, labels, rate)

ImportError: Failed to import any of the following Qt binding modules: PyQt6, PySide6, PyQt5, PySide2